# 1. Dependencies

In [260]:
import warnings
warnings.filterwarnings('ignore')

In [261]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle
import cv2 as cv
import matplotlib.pyplot as plt
import praw
import requests
import json
import csv
import time
import datetime


# 2. Data Acquisation

#### List of Properties of a submission, I am manually eliminating few of them, bacuase flair of the submission is least (or not) dependent of them ie: Flair is rarely dependent on author name, or subreddit etc

##### All Properties: dict_keys(['all_awardings', 'allow_live_comments', 'author', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_text', 'author_flair_type', 'author_fullname', 'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post', 'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video', 'link_flair_background_color', 'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id', 'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls', 'removed_by_category', 'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler', 'steward_reports', 'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title', 'total_awards_received', 'url', 'whitelist_status', 'wls'])


##### Ref: https://praw.readthedocs.io/en/latest/index.html

In [279]:
# creating a reddit app instance to collect post data from r/india submreddit
reddit = praw.Reddit(client_id='xVeb5-ej49aBDg', client_secret='HLuuBB0e1upw1pbZ-oUFtrBplFY', user_agent='reddit-scrap', username='macabdul9', password='Sudo$0#1')

In [280]:
subm = reddit.submission(id="fxqifi")

In [282]:
subm.title

'RSS says Tablighi Jamaat conduct not reflection on all Muslims, they’re aiding govt in fight'

In [281]:
import time

##### Turn Around Time(TAT) for getting all comments of a post is way higher using praw with submission_id than pushshift.io api comment search below code snippets illustrate the same

In [265]:
# start_time = time.time()
# comment_ids = []
# count = 0
# for comment in subm.comments.list():
#     comment_ids.append(comment.id)
#     print(comment.body)
#     count+=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

In [266]:
# start_time = time.time()
# comments = getPushshiftSubData(','.join(comment_ids))
# count = 0
# for comment in comments:
#     print(comment['body'])
#     count +=1
# print("--- %s seconds ---" % (time.time() - start_time))
# print(count)

In [286]:
def get_comments(url, num_comments):
    # get top 20 comments if total comments are more than that
    subm = reddit.submission(url=url)

    # comments = subm.comments.list()
    # if len(comments) > 10:
    #     comments = comments[:10]

    comments_body = []
    sum_score = 0
    count = 0
    if num_comments > 0:
        for i, comment in enumerate(subm.comments.list()):

            # There exist some comments which has no body
            try : 
                comments_body.append(comment.body)
            except:
                comments_body.append('')

            # There exist some comments which has not given a score
            try:
                sum_score += comment.score
            except:
                sum_score += 0            
            count += 1
            # We only need 10 comments
            if (i+1)%10==0:
                break
    try:
        mean = sum_score/count
    except:
        mean = 0.0
    

    return " ".join(comments_body), mean

In [287]:
# get_comments('https://www.reddit.com/r/india/comments/d0ytzc/nasa_on_twitter_space_is_hard_we_commend_isro_s/')

In [288]:
def getPushshiftSubData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=2000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [289]:
features = [
    'allow_live_comments', 'author', 'author_premium', 'can_mod_post', 'contest_mode', 'created_utc',
    'full_link', 'id', 'is_crosspostable', 'is_meta', 'is_original_content', 'is_self', 'is_video',
    'link_flair_text', 'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts', 'over_18',
    'parent_whitelist_status', 'permalink', 'pinned', 'score', 'selftext', 'send_replies', 'spoiler',
    'stickied', 'title', 'total_awards_received', 'url'
]
custom_features = [
    'comments', 'mean_comment_score'
]

In [290]:
def collectSubData(subm):
    
    # extracting all information about a submission
    feature_values = [subm.get(key) for key in features]    
    comments, mean_comment_score = get_comments(subm.get('full_link'), subm.get('num_comments')) 
    feature_values.append(comments)
    feature_values.append(mean_comment_score)
    subStats[subm.get('id')] = feature_values

In [291]:
#Subreddit to query
sub='india'

#before and after dates
after = "1583020800"  # Sun, 01 Mar 2020 00:00:00 
before = "1586563199" # Fri, 10 Apr 2020 23:59:59  
query = "" # title should have either null string more 
subCount = 0
subStats = {}

In [293]:
data = getPushshiftSubData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    # print(len(data))
    if subCount % 1000 == 0:
        print(f'{subCount} submission collected')
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftSubData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1585371364&before=1586563199&subreddit=india
2020-03-30 09:06:20
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1585539380&before=1586563199&subreddit=india
2020-04-01 12:33:21
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1585724601&before=1586563199&subreddit=india
2020-04-03 11:49:35
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1585894775&before=1586563199&subreddit=india
2020-04-05 13:10:19
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1586072419&before=1586563199&subreddit=india
2020-04-07 03:33:17
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1586210597&before=1586563199&subreddit=india
2020-04-09 19:18:54
https://api.pushshift.io/reddit/search/submission/?title=&size=2000&after=1586440134&before=1586563199&subreddit=india
2020-04-11 05:25:19
https://api.pushshift.io/re

ConnectionError: HTTPSConnectionPool(host='api.pushshift.io', port=443): Max retries exceeded with url: /reddit/search/submission/?title=&size=2000&after=1586562919&before=1586563199&subreddit=india (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001612ABBF0B8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [294]:
subCount

20760

In [295]:
with open("SubmissionS-March-2020.pkl", "wb") as f:
    pickle.dump(subStats, f)

In [ ]:
df = pd.DataFrame(data=list(subStats.values()), columns = features + custom_features, index=None)

In [296]:
df.to_csv("SubmissionS-March-2020.csv", index=False)